In [2]:
import numpy as np
import re
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [3]:
with open('tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

# Load label mapping
with open('label_mapping.pkl', 'rb') as mapping_file:
    label_mapping = pickle.load(mapping_file)

c:\ProgramData\Anaconda11\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
model = load_model('gru_rnn.keras')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 46, 50)         │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 46, 150)        │        90,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 46, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 100)            │        75,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,000,411 (7.63 MB)

 Trainable params: 666,803 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,333,608 (5.09 MB)

In [5]:
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Lemmatization and stopword removal
    return ' '.join(tokens)

In [7]:
sample_input="Good service need to increase accuracy"

In [8]:
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
with open('english_stopwords.txt', 'w') as f:
    for word in stop_words:
        f.write(f"{word}\n")

[nltk_data] Downloading package stopwords to C:\Users\Harish's Book2
[nltk_data]     Pro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
preprocessed_input = preprocess_text(sample_input)

input_sequence = tokenizer.texts_to_sequences([preprocessed_input])

input_padded = pad_sequences(input_sequence, maxlen=60, padding='post')

In [10]:
from sklearn.preprocessing import LabelEncoder
with open('label_mapping.pkl', 'rb') as file:
    label_encoder = pickle.load(file)
predictions = model.predict(input_padded)
predicted_class = np.argmax(predictions, axis=-1)
inverse_label_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
predicted_label = inverse_label_mapping[predicted_class[0]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step


In [11]:
predicted_probabilities = predictions[0]

In [12]:
print(f"Input: {sample_input}")
print(f"Predicted class: {predicted_label}")
print(f"Predicted probabilities: {predicted_probabilities}")
print(f"Good: {predicted_probabilities[2]:.4f}, Bad: {predicted_probabilities[0]:.4f}, Neutral: {predicted_probabilities[1]:.4f}")

Input: Good service need to increase accuracy
Predicted class: good
Predicted probabilities: [0.00881643 0.9676703  0.02351336]
Good: 0.0235, Bad: 0.0088, Neutral: 0.9677
